# Encontro 03

## Materiais de apoio:

* The Scientist and Engineer's Guide to Digital Signal Processing, de By Steven W. Smith, Ph.D. (capítulos 19 a 21)
	* Disponível em http://www.dspguide.com

## Tópicos Abordados:

* Filtros de Resposta Infinita ao Impulso (IIR):
  * Butterworth;
  * Chebyshev I e II;
  * Cauer/Elíptico;
  * Bessel/Thomson.

* Importância da resposta impulsiva e da convolução:
  * Coeficientes e cálculo de filtros FIR;
  * Coeficientes e cálculo de filtros IIR.

* Escolha de filtros:
  * FIR ou IIR;
  * Tamanho e atraso de filtros digitais;
  * Estabilidade de filtros digitais; 
  * Características da resposta desejada:
    * Ondulações na banda de passagem e banda de rejeição;
    * Taxa de decaimento do filtro;
    * Fase nula, linear e não-linear.

* Exemplos de aplicações de filtros digitais:
  * Trem de pulsos com ruído;
  * Analisador de harmônicos em rede elétrica.

* Razões para análise em frequência e processamento de sinais:
  * Natureza oscilatória;
  * Informações ocultas em oscilações (frequências);
  * Estudo e reprodução de fenômenos e estruturas naturais;
  * Modelagem matemática de fenômenos oscilatórios (função seno e função cosseno).

## Elaboração:
* Eng. Rodrigo de Marca França.


In [ ]:
# Importação de módulos e instalação de bibliotecas adicionais

# Importação do pacote matématico Math
import math

# Importação do pacote Pandas
import pandas as pd

# Importação dos pacotes NumPy e SciPy
import numpy as np
from scipy.interpolate import interp1d
from scipy import signal

# Importação do pacote PyPlot do MatPlotLib
import matplotlib.pyplot as plt

# Instalação e importação do pacote mpld3
!pip install mpld3
import mpld3

# importação do módulo timeit
import timeit

     |████████████████████████████████| 890kB 8.7MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.2-cp37-none-any.whl size=200617 sha256=d71bd971bc724e9b0c795920e22e0c3a7191d702d19cd3178475dc841d8e5bc4
  Stored in directory: /root/.cache/pip/wheels/21/73/06/ea4b85609301850b1289a282852d92e22fcbf7a250ed5f547f
Successfully built mpld3


In [ ]:
# Diretiva do Notebook para exibição de gráficos inline
%matplotlib inline

# Configuração do tamanho dos gráficos
plt.rcParams["figure.figsize"] = (20,10)

##Filtros de Resposta Infinita ao Impulso (IIR)

Alguns exemplos mais comuns de filtros IIR são os filtros de Butterworth, Chebyshev (tipos I e II), Elíptico e de Bessel.

O filtro de Butterworth é um dos filtros analógicos mais comuns e é caracterizado por ser maximamente plana. Ou seja, esse filtro busca ter a banda de passagem e rejeição com a menor oscilação possível, ao custo de taxa de decaimento.

Possui grande simplicidade e velocidade de execução, tendo grande aplicação tanto analogicamente quanto digitalmente.

In [ ]:
# Criação de um filtro de Butterworth e resposta ao degrau

# Caracteristicas dos filtros
filter_taps = 4
cutout_freq_hz = 200
bandwidth_freq_hz = np.array([150, 250])
sampling_freq_hz = 5000
passband_ripple_db = 5
stopband_ripple_db = 40

# Geração dos coeficientes do filtro de Butterworth
#filter_coefficients_butterworth = signal.iirfilter(filter_taps, bandwidth_freq_hz, btype='bandpass', analog=False, ftype='butter', fs=sampling_freq_hz, output='ba')
filter_coefficients_butterworth = signal.iirfilter(filter_taps, cutout_freq_hz, btype='lowpass', analog=False, ftype='butter', fs=sampling_freq_hz, output='ba')
#filter_coefficients_butterworth = signal.iirfilter(filter_taps, cutout_freq_hz, btype='highpass', analog=False, ftype='butter', fs=sampling_freq_hz, output='ba')
#filter_coefficients_butterworth = signal.iirfilter(filter_taps, bandwidth_freq_hz, btype='bandstop', analog=False, ftype='butter', fs=sampling_freq_hz, output='ba')

# Geração da resposta em frequência do filtro
frequency_response_samples = 1024
filter_response_butterworth = signal.freqz(filter_coefficients_butterworth[0], filter_coefficients_butterworth[1], frequency_response_samples, fs=sampling_freq_hz)

# Geração de um degrau unitário
step_time = np.linspace(0, 1, frequency_response_samples, endpoint=False)
step_signal = (1 - signal.square(2*np.pi*step_time)) / 2

# Geração da resposta ao degrau do filtro
step_signal_filtered = signal.lfilter(filter_coefficients_butterworth[0], filter_coefficients_butterworth[1], step_signal)

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.semilogx(filter_response_butterworth[0], 20*np.log10(np.maximum(np.absolute(filter_response_butterworth[1]), 1e-5)), "-b", label='Butterworth')
plt.legend()
plt.title('Magnitudes em frequência do filtros de Butterworth')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.semilogx(filter_response_butterworth[0], np.maximum(np.absolute(filter_response_butterworth[1]), 1e-5), "-b", label='Butterworth')
plt.legend()
plt.title('Magnitudes em frequência dos filtros de Butterworth')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [Hz]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(step_time, step_signal_filtered, "-b", label='Butterworth')
plt.legend()
plt.title('Resposta ao pulso unitário do Butterworth')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.semilogx(filter_response_butterworth[0], np.angle(filter_response_butterworth[1]), "-b", label='Butterworth')
plt.legend()
plt.title('Fases em frequência dos filtros de Butterworth')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')


# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

O filtro de Chebyshev é outro filtro analógico comumente usado e é caracterizado por uma resposta com ondulações na banda de passagem (Tipo I) ou na banda de rejeição (Tipo II). Ao introduzir ondulações em sua resposta, esse filtro consegue melhorar sua taxa de decaimento. Quanto maior a taxa de decaimento, maiores as ondulações, de forma que o filtro deve ser projetado para equilibrar esses dois parâmetros. Uma ondulação de 0,5% é usualmente uma boa escolha para filtros digitais.

In [ ]:
# Criação de um filtro de Chebyshev I e resposta ao degrau

# Caracteristicas dos filtros
filter_order = 4
cutout_freq_hz = 200
bandwidth_freq_hz = np.array([150, 250])
sampling_freq_hz = 5000
passband_ripple_db = 5
stopband_ripple_db = 40

# Geração dos coeficientes do filtro de Chebyshev I
#filter_coefficients_chebyshev_i = signal.iirfilter(filter_order, bandwidth_freq_hz, rp=passband_ripple_db, btype='bandpass', analog=False, ftype='cheby1', fs=sampling_freq_hz, output='ba')
filter_coefficients_chebyshev_i = signal.iirfilter(filter_order, cutout_freq_hz, rp=passband_ripple_db, btype='lowpass', analog=False, ftype='cheby1', fs=sampling_freq_hz, output='ba')
#filter_coefficients_chebyshev_i = signal.iirfilter(filter_order, cutout_freq_hz, rp=passband_ripple_db, btype='highpass', analog=False, ftype='cheby1', fs=sampling_freq_hz, output='ba')
#filter_coefficients_chebyshev_i = signal.iirfilter(filter_order, bandwidth_freq_hz, rp=passband_ripple_db, btype='bandstop', analog=False, ftype='cheby1', fs=sampling_freq_hz, output='ba')

# Geração da resposta em frequência do filtro
frequency_response_samples = 1024
filter_response_chebyshev_i = signal.freqz(filter_coefficients_chebyshev_i[0], filter_coefficients_chebyshev_i[1], frequency_response_samples, fs=sampling_freq_hz)

# Geração de um degrau unitário
step_time = np.linspace(0, 1, frequency_response_samples, endpoint=False)
step_signal = (1 - signal.square(2*np.pi*step_time)) / 2

# Geração da resposta ao degrau do filtro
step_signal_filtered = signal.lfilter(filter_coefficients_chebyshev_i[0], filter_coefficients_chebyshev_i[1], step_signal)

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.semilogx(filter_response_chebyshev_i[0], 20*np.log10(np.maximum(np.absolute(filter_response_chebyshev_i[1]), 1e-5)), "-b", label='Chebyshev I')
plt.legend()
plt.title('Magnitudes em frequência do filtros de Chebyshev I')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.semilogx(filter_response_chebyshev_i[0], np.maximum(np.absolute(filter_response_chebyshev_i[1]), 1e-5), "-b", label='Chebyshev I')
plt.legend()
plt.title('Magnitudes em frequência dos filtros de Chebyshev I')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [Hz]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(step_time, step_signal_filtered, "-b", label='Chebyshev I')
plt.legend()
plt.title('Resposta ao pulso unitário do Chebyshev I')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.semilogx(filter_response_chebyshev_i[0], np.angle(filter_response_chebyshev_i[1]), "-b", label='Chebyshev I')
plt.legend()
plt.title('Fases em frequência dos filtros de Chebyshev I')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')


# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [ ]:
# Criação de um filtro de Chebyshev II e resposta ao degrau

# Caracteristicas dos filtros
filter_order = 4
cutout_freq_hz = 200
bandwidth_freq_hz = np.array([150, 250])
sampling_freq_hz = 5000
passband_ripple_db = 5
stopband_ripple_db = 40

# Geração dos coeficientes do filtro de Chebyshev II
#filter_coefficients_chebyshev_ii = signal.iirfilter(filter_order, bandwidth_freq_hz, rs=stopband_ripple_db, btype='bandpass', analog=False, ftype='cheby2', fs=sampling_freq_hz, output='ba')
filter_coefficients_chebyshev_ii = signal.iirfilter(filter_order, cutout_freq_hz, rs=stopband_ripple_db, btype='lowpass', analog=False, ftype='cheby2', fs=sampling_freq_hz, output='ba')
#filter_coefficients_chebyshev_ii = signal.iirfilter(filter_order, cutout_freq_hz, rs=stopband_ripple_db, btype='highpass', analog=False, ftype='cheby2', fs=sampling_freq_hz, output='ba')
#filter_coefficients_chebyshev_ii = signal.iirfilter(filter_order, bandwidth_freq_hz, rs=stopband_ripple_db, btype='bandstop', analog=False, ftype='cheby2', fs=sampling_freq_hz, output='ba')

# Geração da resposta em frequência do filtro
frequency_response_samples = 1024
filter_response_chebyshev_ii = signal.freqz(filter_coefficients_chebyshev_ii[0], filter_coefficients_chebyshev_ii[1], frequency_response_samples, fs=sampling_freq_hz)

# Geração de um degrau unitário
step_time = np.linspace(0, 1, frequency_response_samples, endpoint=False)
step_signal = (1 - signal.square(2*np.pi*step_time)) / 2

# Geração da resposta ao degrau do filtro
step_signal_filtered = signal.lfilter(filter_coefficients_chebyshev_ii[0], filter_coefficients_chebyshev_ii[1], step_signal)

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.semilogx(filter_response_chebyshev_ii[0], 20*np.log10(np.maximum(np.absolute(filter_response_chebyshev_ii[1]), 1e-5)), "-b", label='Chebyshev II')
plt.legend()
plt.title('Magnitudes em frequência do filtros de Chebyshev II')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.semilogx(filter_response_chebyshev_ii[0], np.maximum(np.absolute(filter_response_chebyshev_ii[1]), 1e-5), "-b", label='Chebyshev II')
plt.legend()
plt.title('Magnitudes em frequência dos filtros de Chebyshev II')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [Hz]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(step_time, step_signal_filtered, "-b", label='Chebyshev II')
plt.legend()
plt.title('Resposta ao pulso unitário do Chebyshev II')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.semilogx(filter_response_chebyshev_ii[0], np.angle(filter_response_chebyshev_ii[1]), "-b", label='Chebyshev II')
plt.legend()
plt.title('Fases em frequência dos filtros de Chebyshev II')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')


# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

O filtro Elíptico (ou de Cauer) é um filtro analógico que consegue obter altas taxas de decaimento permitindo ondulações tanto na banda de passagem quanto na banda de rejeição. Esses filtros permitem as melhoras taxas de decaimento para um mesmo número de coeficientes.

In [ ]:
# Criação de um filtro de Cauer/Elliptic e resposta ao degrau

# Caracteristicas dos filtros
filter_order = 4
cutout_freq_hz = 200
bandwidth_freq_hz = np.array([150, 250])
sampling_freq_hz = 5000
passband_ripple_db = 5
stopband_ripple_db = 40

# Geração dos coeficientes do filtro de Cauer/Elliptic
#filter_coefficients_elliptic = signal.iirfilter(filter_order, bandwidth_freq_hz, rp=passband_ripple_db, rs=stopband_ripple_db, btype='bandpass', analog=False, ftype='ellip', fs=sampling_freq_hz, output='ba')
filter_coefficients_elliptic = signal.iirfilter(filter_order, cutout_freq_hz, rp=passband_ripple_db, rs=stopband_ripple_db, btype='lowpass', analog=False, ftype='ellip', fs=sampling_freq_hz, output='ba')
#filter_coefficients_elliptic = signal.iirfilter(filter_order, cutout_freq_hz, rp=passband_ripple_db, rs=stopband_ripple_db, btype='highpass', analog=False, ftype='ellip', fs=sampling_freq_hz, output='ba')
#filter_coefficients_elliptic = signal.iirfilter(filter_order, bandwidth_freq_hz, rp=passband_ripple_db, rs=stopband_ripple_db, btype='bandstop', analog=False, ftype='ellip', fs=sampling_freq_hz, output='ba')

# Geração da resposta em frequência do filtro
frequency_response_samples = 1024
filter_response_elliptic = signal.freqz(filter_coefficients_elliptic[0], filter_coefficients_elliptic[1], frequency_response_samples, fs=sampling_freq_hz)

# Geração de um degrau unitário
step_time = np.linspace(0, 1, frequency_response_samples, endpoint=False)
step_signal = (1 - signal.square(2*np.pi*step_time)) / 2

# Geração da resposta ao degrau do filtro
step_signal_filtered = signal.lfilter(filter_coefficients_elliptic[0], filter_coefficients_elliptic[1], step_signal)

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.semilogx(filter_response_elliptic[0], 20*np.log10(np.maximum(np.absolute(filter_response_elliptic[1]), 1e-5)), "-b", label='Cauer/Elliptic')
plt.legend()
plt.title('Magnitudes em frequência do filtros de Cauer/Elliptic')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.semilogx(filter_response_elliptic[0], np.maximum(np.absolute(filter_response_elliptic[1]), 1e-5), "-b", label='Cauer/Elliptic')
plt.legend()
plt.title('Magnitudes em frequência dos filtros de Cauer/Elliptic')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [Hz]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(step_time, step_signal_filtered, "-b", label='Cauer/Elliptic')
plt.legend()
plt.title('Resposta ao pulso unitário do Cauer/Elliptic')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.semilogx(filter_response_elliptic[0], np.angle(filter_response_elliptic[1]), "-b", label='Cauer/Elliptic')
plt.legend()
plt.title('Fases em frequência dos filtros de Cauer/Elliptic')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')


# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

O filtro de Bessel (ou Thomson) é um filtro analógico que possui a característica de ter a fase o mais linear possível. Isso o torna muito interessante em aplicações onde a fase dos sinais é crítica. Sua linearidade na fase se faz em troca de taxa de decaimento e oscilações nas bandas de passagem e rejeição. 

In [ ]:
# Criação de um filtro de Bessel/Thomson e resposta ao degrau

# Caracteristicas dos filtros
filter_order = 4
cutout_freq_hz = 200
bandwidth_freq_hz = np.array([150, 250])
sampling_freq_hz = 5000
passband_ripple_db = 5
stopband_ripple_db = 40

# Geração dos coeficientes do filtro de Bessel/Thomson
#filter_coefficients_bessel = signal.iirfilter(filter_order, bandwidth_freq_hz, btype='bandpass', analog=False, ftype='bessel', fs=sampling_freq_hz, output='ba')
filter_coefficients_bessel = signal.iirfilter(filter_order, cutout_freq_hz, btype='lowpass', analog=False, ftype='bessel', fs=sampling_freq_hz, output='ba')
#filter_coefficients_bessel = signal.iirfilter(filter_order, cutout_freq_hz, btype='highpass', analog=False, ftype='bessel', fs=sampling_freq_hz, output='ba')
#filter_coefficients_bessel = signal.iirfilter(filter_order, bandwidth_freq_hz, btype='bandstop', analog=False, ftype='bessel', fs=sampling_freq_hz, output='ba')

# Geração da resposta em frequência do filtro
frequency_response_samples = 1024
filter_response_bessel = signal.freqz(filter_coefficients_bessel[0], filter_coefficients_bessel[1], frequency_response_samples, fs=sampling_freq_hz)

# Geração de um degrau unitário
step_time = np.linspace(0, 1, frequency_response_samples, endpoint=False)
step_signal = (1 - signal.square(2*np.pi*step_time)) / 2

# Geração da resposta ao degrau do filtro
step_signal_filtered = signal.lfilter(filter_coefficients_bessel[0], filter_coefficients_bessel[1], step_signal)

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.semilogx(filter_response_bessel[0], 20*np.log10(np.maximum(np.absolute(filter_response_bessel[1]), 1e-5)), "-b", label='Bessel/Thomson')
plt.legend()
plt.title('Magnitudes em frequência do filtros de Bessel/Thomson')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.semilogx(filter_response_bessel[0], np.maximum(np.absolute(filter_response_bessel[1]), 1e-5), "-b", label='Bessel/Thomson')
plt.legend()
plt.title('Magnitudes em frequência dos filtros de Bessel/Thomson')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [Hz]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(step_time, step_signal_filtered, "-b", label='Bessel/Thomson')
plt.legend()
plt.title('Resposta ao pulso unitário do Bessel/Thomson')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.semilogx(filter_response_bessel[0], np.angle(filter_response_bessel[1]), "-b", label='Bessel/Thomson')
plt.legend()
plt.title('Fases em frequência dos filtros de Bessel/Thomson')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')


# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

## Importância da resposta impulsiva e da convolução:

Quando projetamos e aplicamos um filtro, desejamos obter uma distorção nas frequências de um sinal temporal. Essas distorções podem ser entendidas como uma multiplicação no domínio da frequência do sinal original e uma sequência de ganhos. Como desejamos obter uma reposta temporal do sinal original, temos apenas duas opções:
* Aplicar a DFT no sinal original e aplicar nela os ganhos em frequência do filtro;
* Executar a convolução (operação dual no tempo da multiplicação na frequência) com o sinal original no domínio do tempo e uma caracterização da resposta do filtro (resposta ao impulso).

Ou seja, a convolução com a resposta impulsiva de um filtro nos permite aplicar os ganhos em frequência diretamente na resposta temporal do sinal. Como uma convolução discreta pode ser vista como a aplicação de uma série de ganhos as amostras de um sinal, podemos aplicar um filtro a um sinal temporal ao multiplicar suas amostras pela resposta impulsiva no filtro (na sequência correta).

A grande importância desse resultado é que para aplicar um filtro a qualquer sinal amostrado, precisamos apenas conhecer a resposta impulsiva desse filtro. Essa resposta é usualmente representada pelos coeficientes do filtro. Tendo os coeficientes do filtro, não precisamos de mais nenhuma informação e podemos aplicar o filtro em um número infinito de amostras.

Filtros do tipo FIR possuem respostas finitas que dependem somente de suas entradas. Ou seja, os seus coeficientes podem ser definidos como:
> $a_0, a_1, ..., a_{N-1}$ \\
$y[0] = a_0 x[0] + a_1 x[-1] + ... + a_{N-1} x[N-1]$

Sendo $N$ o tamanho do filtro, $x$ as amostras do sinal original e $y$ as amostras do sinal filtrado.

As funções **firwin** e **firwin2** do pacote SciPy retornam exatamente os coeficientes do filtro FIR gerado.

Filtros do tipo IIR possuem respostas infinitas que dependem de suas entradas e saídas passadas. Ou seja, os seus coeficientes podem ser definidos como:
> $a_0, a_1, ..., a_{N-1}, b_1, ..., b_{N-1}$ \\
$y[0] = a_0 x[0] + a_1 x[-1] + ... + a_{N-1} x[N-1] + b_1 y[-1] + ... + b_{N-1} y[N-1]$

Sendo $N$ o tamanho do filtro, $x$ as amostras do sinal original e $y$ as amostras do sinal filtrado.

A função **iirfilter** do pacote SciPy pode retornar exatamente os coeficientes do filtro IIR gerado.

## Escolha de filtros:

Dadas as diferentes possibilidades de filtros, como escolher o filtro mais apropriado para uma determinada aplicação? Essa é uma pergunta complexa e que não tem uma resposta única.

Usualmente aplicações diferentes precisam de características diferentes, o que envolve usar filtros diferentes. Aplicações comuns (como processamento de áudio, imagem, sinais biológicos, etc.) são alvo de diversos estudos, de forma que já existem as recomendações de como tratar e filtras esses sinais.

Mas e se o sinal é desconhecido ou não é uma aplicação comum? O filtro final a ser aplicado provavelmente vai se o resultado de diversas tentativas e erros por parte do projetista. No entanto, deve-se manter algumas coisas importantes em mente na hora de escolher um filtro:
* Tipo da resposta ao impulso (FIR x IIR):
  * Filtros FIR costumam a permitir respostas mais bem comportadas e mais previsíveis, além de serem mais estáveis e possuírem fase linear. No entanto, eles exigem filtros maiores (com mais coeficientes) para obter respostas similares os filtros IIR. São uma ótima escolha quando se deseja performance na manipulação das frequências do sinal.
  * Filtros IIR geralmente são criados através da desratização de filtros analógicos e conseguem excelentes respostas mesmo com tamanhos reduzidos. Em troca, eles costumam a apresentam respostas menos comportadas, maior instabilidade e fase não linear. São uma ótima escolha quando se deseja performance computacional na manipulação da resposta temporal do sinal.

* Tamanho e atraso de filtros digitais:
  * A complexidade computacional de um filtro, assim como seu atraso depende diretamente de seu tamanho (quantidade de coeficientes);
  * Por exemplo, um filtro FIR com 128 coeficientes vai precisar executar pelo menos 128 multiplicações e adições, além de precisar de 128 amostras do sinal para calcular um único valor de saída. Isso significa que um evento que ocorra no sinal original só será completamente reproduzido no sinal filtrado após 128 amostras e 128 tempos de amostragem;
  * Se o seu período de amostragem é de um 1 segundo, por exemplo, então esse evento só será devidamente reproduzido 128 segundos depois de ele ocorrer. Esse pode ser definido como o atraso desse filtro e pode ser significativo ou não, dependo da sua aplicação. 

* Estabilidade de filtros digitais:
  * Podemos definir a estabilidade de um filtro como sua capacidade de convergir para um valor estável e finito. Ou seja, um filtro estável consegue convergir para um valor finito dentro de um tempo finito. Por outro lado, um filtro instável não consegue convergir para um valor finito, gerando um sinal que cresce ou oscila infinitamente;
  * De uma maneira generalizada, a estabilidade de um filtro está relacionada ao seu tamanho. Quanto maior um filtro, maior a sua tendência de ser instável;
  * Filtros FIR costumam a ser altamente estáveis, mesmo com grandes tamanhos. É isso que permite a eles obterem respostas excelentes em frequência, pois seus tamanhos podem ser aumentados indefinidamente;
  * Filtros IIR, por outro lado, tendem a ser instáveis. Isso se deve a uma combinação da desratização do filtro analógico original (geralmente por transformação bilinear) com erros numéricos e de arredondamento por conta da representação binaria. Para garantir a estabilidade, geralmente se utilizam filtros de pequeno tamanho. 

* Fase nula, linear e não-linear:
  * A fase de um filtro define como as fases do sinal original serão modificadas;
  * Um filtro de fase nula não altera as fases do sinal original, mas é difícil de ser realizado em situações práticas;
  * Um filtro com fase linear altera linearmente as fases do sinal original e é facilmente realizável com filtros FIR. Uma alteração linear das fazes causa pouca deformação no sinal original e deformações simétricas, sendo desejado quando a fase do sinal é um dado importante;
  * Um filtro não-linear altera de maneira não-linear as fases do sinal original e comumente ocorre em filtros IIR. Essa alteração não-linear pode causar deformações significativas e não simétricas no sinal original. É muito comum sinais onde a informação de fase não é particularmente importante, de forma que essa alteração não-linear na fase é praticamente irrelevante.


## Exemplos de aplicações de filtros digitais:

Exemplo 1 - Trem de pulsos com ruído

In [ ]:
# Criação de um trem de pulsos com ruído

# Geração de um trem de pulsos
pulse_samples = 1024
pulse_time = np.linspace(0, 5.5, pulse_samples, endpoint=False)
pulse_signal = (1 - signal.square(2*np.pi*pulse_time, 0.75)) / 2

# Adição de ruído ao sinal
noise_level = 0.5
pulse_signal = pulse_signal + noise_level*np.random.rand(pulse_samples)

# Criação da figura
fig = plt.figure()
plt.step(pulse_time, pulse_signal, "-b")
plt.title('Trem de pulsos com ruído')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [ ]:
# Filtragem do trem de pulso com ruído usando um filtro de média móvel

# Geração dos coeficientes do filtro de média móvel
filter_taps = 16
filter_coefficients_moving_avg = np.ones(filter_taps)/filter_taps

# Aplicação do filtro no trem de pulso
pulse_train_response_moving_avg = signal.lfilter(filter_coefficients_moving_avg, 1.0, pulse_signal)

# Criação da figura
fig = plt.figure()
plt.step(pulse_time, pulse_train_response_moving_avg, "-b")
#plt.plot(pulse_time[filter_taps:], pulse_train_response_moving_avg[filter_taps:], "-b")
plt.title('Trem de pulsos com ruído')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

Exemplo 2 - Analisador de harmônicos em rede elétrica


In [ ]:
# Criação de um sinal amostrado de Tensão de uma Rede Elétrica Residencial com harmônicos

# Definição das características do sinal
num_signal = 1024
num_samples = 256
measurement_noise = 0.01
quantization = 4096
quantization_limits = np.array([-250, 250])
sampling_rate_hz = 60*5*5

# Criação do vetor de tempo e vetores de dados
time_signal = np.linspace(0, num_samples/sampling_rate_hz, num_signal)
time_samples = np.linspace(0, num_samples/sampling_rate_hz, num_samples + 1)[:-1]
#voltage_signal = 110*np.sqrt(2)*np.sin(2*math.pi*60*time_signal + math.pi/4)
#voltage_samples = 110*np.sqrt(2)*np.sin(2*math.pi*60*time_samples + math.pi/4)
voltage_signal = 10 + 110*np.sqrt(2)*np.sin(2*math.pi*60*time_signal + math.pi/4) + 110/5*np.sqrt(2)*np.sin(2*math.pi*2*60*time_signal - math.pi/4) + 110/10*np.sqrt(2)*np.sin(2*math.pi*3*60*time_signal - math.pi/4) + 110/15*np.sqrt(2)*np.sin(2*math.pi*4*60*time_signal - math.pi/4) + 110/20*np.sqrt(2)*np.sin(2*math.pi*5*60*time_signal - math.pi/4)
voltage_samples = 10 + 110*np.sqrt(2)*np.sin(2*math.pi*60*time_samples + math.pi/4) + 110/5*np.sqrt(2)*np.sin(2*math.pi*2*60*time_samples - math.pi/4) + 110/10*np.sqrt(2)*np.sin(2*math.pi*3*60*time_samples - math.pi/4) + 110/15*np.sqrt(2)*np.sin(2*math.pi*4*60*time_samples - math.pi/4) + 110/20*np.sqrt(2)*np.sin(2*math.pi*5*60*time_samples - math.pi/4)
voltage_samples = voltage_samples + voltage_samples*np.random.rand(voltage_samples.size)*measurement_noise

# Efeito da quantização
quantized_samples = np.round(voltage_samples*quantization/(quantization_limits[1] - quantization_limits[0]))
quantized_voltage_samples = quantized_samples * (quantization_limits[1] - quantization_limits[0]) / quantization;

# Criação da figura
fig = plt.figure()
#plt.plot(time_signal, voltage_signal, '-b')
#plt.plot(time_samples, quantized_voltage_samples, ',r')
plt.step(time_samples, quantized_voltage_samples, 'r')
plt.title('Tensão da Rede Eletrica Residencial')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [ ]:
# Aplicação da FFT com janelamento na Tensão de uma Rede Elétrica Residencial com harmônicos

# Aplicação da janela
quantized_voltage_samples_windowed = quantized_voltage_samples * signal.hann(quantized_voltage_samples.size)
#quantized_voltage_samples_windowed = quantized_voltage_samples * signal.hamming(quantized_voltage_samples.size)
#quantized_voltage_samples_windowed = quantized_voltage_samples * signal.blackman(quantized_voltage_samples.size)
#quantized_voltage_samples_windowed = quantized_voltage_samples * signal.flattop(quantized_voltage_samples.size)

voltage_signal_windowed = voltage_signal * signal.hann(voltage_signal.size)
#voltage_signal_windowed = voltage_signal * signal.hamming(voltage_signal.size)
#voltage_signal_windowed = voltage_signal * signal.blackman(voltage_signal.size)
#voltage_signal_windowed = voltage_signal * signal.flattop(voltage_signal.size)

# Execução do algoritmo da FFT
fft_windowed = np.fft.fft(quantized_voltage_samples_windowed, num_samples)

# Geração do vetor de frequências da FFT
freqs_windowed = np.fft.fftfreq(num_samples)

# Shift da FFT e do vetor de frequências
fft_windowed_shifted = np.fft.fftshift(fft_windowed)
freqs_windowed_shifted = np.fft.fftshift(freqs_windowed)

# Corrige o valor da FFT
fft_windowed_scaled = fft_windowed_shifted/num_samples
freqs_windowed_shifted_hz = freqs_windowed_shifted * sampling_rate_hz

# Seleciona a FFT a ser plotada
#fft_windowed_plot = fft_windowed
#fft_windowed_plot = fft_windowed_shifted
fft_windowed_plot = fft_windowed_scaled

# Seleciona o vetor de frequências ser plotado
#freqs_windowed_plot = freqs_windowed
#freqs_windowed_plot = freqs_windowed_shifted
freqs_windowed_plot = freqs_windowed_shifted_hz

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
stem_fft_windowed_real = plt.stem(freqs_windowed_shifted_hz, np.absolute(fft_windowed_plot), use_line_collection=True)
stem_fft_windowed_real[0].set_markerfacecolor('none')
stem_fft_windowed_real[0].set_markersize(2)
plt.title('Magnitudes da FFT da Tensão Elétrica Residencial Janelada')
plt.ylabel('Amplitude [V]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 1, 2)
stem_fft_windowed_imag = plt.stem(freqs_windowed_shifted_hz, np.angle(fft_windowed_plot), use_line_collection=True)
stem_fft_windowed_imag[0].set_markerfacecolor('none')
stem_fft_windowed_imag[0].set_markersize(2)
plt.title('Fases da FFT da Tensão Elétrica Residencial Janelada')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
labels_fft_windowed_real = ["Freq {:.3f} = {:.3f}".format(freqs_windowed_shifted_hz[i], np.absolute(fft_windowed_plot[i])) for i in range(freqs_windowed_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_windowed_real[0], labels_fft_windowed_real))
labels_fft_windowed_imag = ["Freq {:.3f} = {:.3f}".format(freqs_windowed_shifted_hz[i], np.angle(fft_windowed_plot[i])) for i in range(freqs_windowed_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_windowed_imag[0], labels_fft_windowed_imag))
mpld3.display()

In [ ]:
# Criação de filtros para separação dos harmônicos do sinal de tensão

# Caracteristicas dos filtros
filter_order = 4

# Geração do filtro para separação do nível médio
h0_cutout_freq_hz = 15
h0_filter_coefficients = signal.iirfilter(filter_order, h0_cutout_freq_hz, btype='lowpass', analog=False, ftype='butter', fs=sampling_rate_hz, output='ba')

# Geração do filtro para a primeira harmônica
h1_bandwidth_freq_hz = np.array([15, 75])
h1_filter_coefficients = signal.iirfilter(filter_order, h1_bandwidth_freq_hz, btype='bandpass', analog=False, ftype='butter', fs=sampling_rate_hz, output='ba')

# Geração do filtro para a segunda harmônica
h2_bandwidth_freq_hz = np.array([105, 135])
h2_filter_coefficients = signal.iirfilter(filter_order, h2_bandwidth_freq_hz, btype='bandpass', analog=False, ftype='butter', fs=sampling_rate_hz, output='ba')

# Geração do filtro para a terceira harmônica
h3_bandwidth_freq_hz = np.array([165, 195])
h3_filter_coefficients = signal.iirfilter(filter_order, h3_bandwidth_freq_hz, btype='bandpass', analog=False, ftype='butter', fs=sampling_rate_hz, output='ba')

# Geração do filtro para a quarta harmônica
h4_bandwidth_freq_hz = np.array([225, 255])
h4_filter_coefficients = signal.iirfilter(filter_order, h4_bandwidth_freq_hz, btype='bandpass', analog=False, ftype='butter', fs=sampling_rate_hz, output='ba')

# Geração do filtro para a quinta harmônica
h5_bandwidth_freq_hz = np.array([285, 315])
h5_filter_coefficients = signal.iirfilter(filter_order, h5_bandwidth_freq_hz, btype='bandpass', analog=False, ftype='butter', fs=sampling_rate_hz, output='ba')

# Geração da resposta em frequência dos filtro
frequency_response_samples = 1024
h0_filter_response = signal.freqz(h0_filter_coefficients[0], h0_filter_coefficients[1], frequency_response_samples, fs=sampling_rate_hz)
h1_filter_response = signal.freqz(h1_filter_coefficients[0], h1_filter_coefficients[1], frequency_response_samples, fs=sampling_rate_hz)
h2_filter_response = signal.freqz(h2_filter_coefficients[0], h2_filter_coefficients[1], frequency_response_samples, fs=sampling_rate_hz)
h3_filter_response = signal.freqz(h3_filter_coefficients[0], h3_filter_coefficients[1], frequency_response_samples, fs=sampling_rate_hz)
h4_filter_response = signal.freqz(h4_filter_coefficients[0], h4_filter_coefficients[1], frequency_response_samples, fs=sampling_rate_hz)
h5_filter_response = signal.freqz(h5_filter_coefficients[0], h5_filter_coefficients[1], frequency_response_samples, fs=sampling_rate_hz)

# Geração de um degrau unitário
step_time = np.linspace(0, 1, frequency_response_samples, endpoint=False)
step_signal = (1 - signal.square(2*np.pi*step_time)) / 2

# Geração da resposta ao degrau dos filtros
step_signal_h0_filtered = signal.lfilter(h0_filter_coefficients[0], h0_filter_coefficients[1], step_signal)
step_signal_h1_filtered = signal.lfilter(h1_filter_coefficients[0], h1_filter_coefficients[1], step_signal)
step_signal_h2_filtered = signal.lfilter(h2_filter_coefficients[0], h2_filter_coefficients[1], step_signal)
step_signal_h3_filtered = signal.lfilter(h3_filter_coefficients[0], h3_filter_coefficients[1], step_signal)
step_signal_h4_filtered = signal.lfilter(h4_filter_coefficients[0], h4_filter_coefficients[1], step_signal)
step_signal_h5_filtered = signal.lfilter(h5_filter_coefficients[0], h5_filter_coefficients[1], step_signal)

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.semilogx(h0_filter_response[0], 20*np.log10(np.maximum(np.absolute(h0_filter_response[1]), 1e-5)), "-b", label='H0')
plt.semilogx(h1_filter_response[0], 20*np.log10(np.maximum(np.absolute(h1_filter_response[1]), 1e-5)), "--g", label='H1')
plt.semilogx(h2_filter_response[0], 20*np.log10(np.maximum(np.absolute(h2_filter_response[1]), 1e-5)), "-.r", label='H2')
plt.semilogx(h3_filter_response[0], 20*np.log10(np.maximum(np.absolute(h3_filter_response[1]), 1e-5)), "-k", label='H3')
plt.semilogx(h4_filter_response[0], 20*np.log10(np.maximum(np.absolute(h4_filter_response[1]), 1e-5)), "--m", label='H4')
plt.semilogx(h5_filter_response[0], 20*np.log10(np.maximum(np.absolute(h5_filter_response[1]), 1e-5)), "-.y", label='H5')
plt.legend()
plt.title('Magnitudes em frequência dos filtros projetados')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.semilogx(h0_filter_response[0], np.maximum(np.absolute(h0_filter_response[1]), 1e-5), "-b", label='H0')
plt.semilogx(h1_filter_response[0], np.maximum(np.absolute(h1_filter_response[1]), 1e-5), "--g", label='H1')
plt.semilogx(h2_filter_response[0], np.maximum(np.absolute(h2_filter_response[1]), 1e-5), "-.r", label='H2')
plt.semilogx(h3_filter_response[0], np.maximum(np.absolute(h3_filter_response[1]), 1e-5), "-k", label='H3')
plt.semilogx(h4_filter_response[0], np.maximum(np.absolute(h4_filter_response[1]), 1e-5), "--m", label='H4')
plt.semilogx(h5_filter_response[0], np.maximum(np.absolute(h5_filter_response[1]), 1e-5), "-.y", label='H5')
plt.legend()
plt.title('Magnitudes em frequência dos filtros projetados')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [Hz]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(step_time, step_signal_h0_filtered, "-b", label='H0')
plt.plot(step_time, step_signal_h1_filtered, "--g", label='H1')
plt.plot(step_time, step_signal_h2_filtered, "-.r", label='H2')
plt.plot(step_time, step_signal_h3_filtered, "-k", label='H3')
plt.plot(step_time, step_signal_h4_filtered, "--m", label='H4')
plt.plot(step_time, step_signal_h5_filtered, "-.y", label='H5')
plt.legend()
plt.title('Resposta ao pulso unitário dos filtros projetados')
plt.ylabel('Amplitude')
plt.xlabel('Tempo')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.semilogx(h0_filter_response[0], np.angle(h0_filter_response[1]), "-b", label='H0')
plt.semilogx(h1_filter_response[0], np.angle(h1_filter_response[1]), "--g", label='H1')
plt.semilogx(h2_filter_response[0], np.angle(h2_filter_response[1]), "-.r", label='H2')
plt.semilogx(h3_filter_response[0], np.angle(h3_filter_response[1]), "-k", label='H3')
plt.semilogx(h4_filter_response[0], np.angle(h4_filter_response[1]), "--m", label='H4')
plt.semilogx(h5_filter_response[0], np.angle(h5_filter_response[1]), "-.y", label='H5')
plt.legend()
plt.title('Fases em frequência dos filtros projetados')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [ ]:
# Aplicação dos filtros no sinal de tensão elétrica para separação dos harmônicos

# Aplicação dos filtros no sinal
voltage_signal_h0_filtered = signal.lfilter(h0_filter_coefficients[0], h0_filter_coefficients[1], quantized_voltage_samples)
voltage_signal_h1_filtered = signal.lfilter(h1_filter_coefficients[0], h1_filter_coefficients[1], quantized_voltage_samples)
voltage_signal_h2_filtered = signal.lfilter(h2_filter_coefficients[0], h2_filter_coefficients[1], quantized_voltage_samples)
voltage_signal_h3_filtered = signal.lfilter(h3_filter_coefficients[0], h3_filter_coefficients[1], quantized_voltage_samples)
voltage_signal_h4_filtered = signal.lfilter(h4_filter_coefficients[0], h4_filter_coefficients[1], quantized_voltage_samples)
voltage_signal_h5_filtered = signal.lfilter(h5_filter_coefficients[0], h5_filter_coefficients[1], quantized_voltage_samples)

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(3, 2, 1)
plt.step(time_samples, voltage_signal_h0_filtered, "-b", label='H0')
plt.title('Tensão da Rede Eletrica Residencial - Nível Médio')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação do segundo subplot
plt.subplot(3, 2, 2)
plt.step(time_samples, voltage_signal_h1_filtered, "--g", label='H1')
plt.title('Tensão da Rede Eletrica Residencial - Primeira Harmônica')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação do terceiro subplot
plt.subplot(3, 2, 3)
plt.step(time_samples, voltage_signal_h2_filtered, "-.r", label='H2')
plt.title('Tensão da Rede Eletrica Residencial - Segunda Harmônica')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação do quarto subplot
plt.subplot(3, 2, 4)
plt.step(time_samples, voltage_signal_h3_filtered, "-k", label='H3')
plt.title('Tensão da Rede Eletrica Residencial - Terceira Harmônica')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação do quinto subplot
plt.subplot(3, 2, 5)
plt.step(time_samples, voltage_signal_h4_filtered, "--m", label='H4')
plt.title('Tensão da Rede Eletrica Residencial - Quarta Harmônica')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação do sexto subplot
plt.subplot(3, 2, 6)
plt.step(time_samples, voltage_signal_h5_filtered, "-.y", label='H5')
plt.title('Tensão da Rede Eletrica Residencial - Quinta Harmônica')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [ ]:
# Cálculo do valor RMS de cada harmônicos do sinal de tensão elétrica

# Geração dos valores RMS medidos
voltage_rms_h0_measure = np.sqrt(np.mean(voltage_signal_h0_filtered ** 2))
voltage_rms_h1_measure = np.sqrt(np.mean(voltage_signal_h1_filtered ** 2))
voltage_rms_h2_measure = np.sqrt(np.mean(voltage_signal_h2_filtered ** 2))
voltage_rms_h3_measure = np.sqrt(np.mean(voltage_signal_h3_filtered ** 2))
voltage_rms_h4_measure = np.sqrt(np.mean(voltage_signal_h4_filtered ** 2))
voltage_rms_h5_measure = np.sqrt(np.mean(voltage_signal_h5_filtered ** 2))

# Geração dos valores RMS reais
voltage_rms_h0_real = 10
voltage_rms_h1_real = 110
voltage_rms_h2_real = 110/5
voltage_rms_h3_real = 110/10
voltage_rms_h4_real = 110/15
voltage_rms_h5_real = 110/20

# Geração dos erros relativos
voltage_rms_h0_errp = abs(voltage_rms_h0_measure - voltage_rms_h0_real)/voltage_rms_h0_real * 100
voltage_rms_h1_errp = abs(voltage_rms_h1_measure - voltage_rms_h1_real)/voltage_rms_h1_real * 100
voltage_rms_h2_errp = abs(voltage_rms_h2_measure - voltage_rms_h2_real)/voltage_rms_h2_real * 100
voltage_rms_h3_errp = abs(voltage_rms_h3_measure - voltage_rms_h3_real)/voltage_rms_h3_real * 100
voltage_rms_h4_errp = abs(voltage_rms_h4_measure - voltage_rms_h4_real)/voltage_rms_h4_real * 100
voltage_rms_h5_errp = abs(voltage_rms_h5_measure - voltage_rms_h5_real)/voltage_rms_h5_real * 100

# Exibição dos valores RMS e erros
print("Valor RMS do Nível Médio       : Medido =", "{:.3f}".format(voltage_rms_h0_measure), " V; Real =" , "{:.3f}".format(voltage_rms_h0_real), " V; Erro% =", "{:.3f}".format(voltage_rms_h0_errp))
print("Valor RMS da Primeira Harmônica: Medido =", "{:.3f}".format(voltage_rms_h1_measure), "V; Real ="  , "{:.3f}".format(voltage_rms_h1_real), "V; Erro% =", "{:.3f}".format(voltage_rms_h1_errp))
print("Valor RMS da Segunda Harmônica : Medido =", "{:.3f}".format(voltage_rms_h2_measure), " V; Real =" , "{:.3f}".format(voltage_rms_h2_real), " V; Erro% =", "{:.3f}".format(voltage_rms_h2_errp))
print("Valor RMS da Terceira Harmônica: Medido =", "{:.3f}".format(voltage_rms_h3_measure), " V; Real =" , "{:.3f}".format(voltage_rms_h3_real), " V; Erro% =", "{:.3f}".format(voltage_rms_h3_errp))
print("Valor RMS da Quarta Harmônica  : Medido =", "{:.3f}".format(voltage_rms_h4_measure), "  V; Real =", "{:.3f}".format(voltage_rms_h4_real), "  V; Erro% =", "{:.3f}".format(voltage_rms_h4_errp))
print("Valor RMS da Quinta Harmônica  : Medido =", "{:.3f}".format(voltage_rms_h5_measure), "  V; Real =", "{:.3f}".format(voltage_rms_h5_real), "  V; Erro% =", "{:.3f}".format(voltage_rms_h5_errp))

Valor RMS do Nível Médio       : Medido = 11.281  V; Real = 10.000  V; Erro% = 12.805
Valor RMS da Primeira Harmônica: Medido = 103.974 V; Real = 110.000 V; Erro% = 5.478
Valor RMS da Segunda Harmônica : Medido = 21.286  V; Real = 22.000  V; Erro% = 3.245
Valor RMS da Terceira Harmônica: Medido = 10.505  V; Real = 11.000  V; Erro% = 4.499
Valor RMS da Quarta Harmônica  : Medido = 6.898   V; Real = 7.333   V; Erro% = 5.937
Valor RMS da Quinta Harmônica  : Medido = 5.156   V; Real = 5.500   V; Erro% = 6.263


In [ ]:
# Aplicação do medidor de harmônicos de maneira recursiva

# Inicialização do vetor de amostras de entradas
samples_amount = 9
sig_samples_in = np.zeros(samples_amount)

# Inicialização do vetor de amostras de saidas
sig_samples_out_h0 = np.zeros(samples_amount)
sig_samples_out_h1 = np.zeros(samples_amount)
sig_samples_out_h2 = np.zeros(samples_amount)
sig_samples_out_h3 = np.zeros(samples_amount)
sig_samples_out_h4 = np.zeros(samples_amount)
sig_samples_out_h5 = np.zeros(samples_amount)

# Inicialização dos vetores de saídas
sig_out_h0 = np.zeros(num_samples)
sig_out_h1 = np.zeros(num_samples)
sig_out_h2 = np.zeros(num_samples)
sig_out_h3 = np.zeros(num_samples)
sig_out_h4 = np.zeros(num_samples)
sig_out_h5 = np.zeros(num_samples)

# Execução do loop amostra por amostra
for sample in range(num_samples):

  # Atualização do vetor de amostras com a amostra mais recente
  sig_samples_in[8] = sig_samples_in[7]
  sig_samples_in[7] = sig_samples_in[6]
  sig_samples_in[6] = sig_samples_in[5]
  sig_samples_in[5] = sig_samples_in[4]
  sig_samples_in[4] = sig_samples_in[3]
  sig_samples_in[3] = sig_samples_in[2]
  sig_samples_in[2] = sig_samples_in[1]
  sig_samples_in[1] = sig_samples_in[0]
  sig_samples_in[0] = quantized_voltage_samples[sample]

  # Atualização dos vetores de saída com as saídas antigas
  sig_samples_out_h0[8] = sig_samples_out_h0[7]
  sig_samples_out_h0[7] = sig_samples_out_h0[6]
  sig_samples_out_h0[6] = sig_samples_out_h0[5]
  sig_samples_out_h0[5] = sig_samples_out_h0[4]
  sig_samples_out_h0[4] = sig_samples_out_h0[3]
  sig_samples_out_h0[3] = sig_samples_out_h0[2]
  sig_samples_out_h0[2] = sig_samples_out_h0[1]
  sig_samples_out_h0[1] = sig_samples_out_h0[0]

  sig_samples_out_h1[8] = sig_samples_out_h1[7]
  sig_samples_out_h1[7] = sig_samples_out_h1[6]
  sig_samples_out_h1[6] = sig_samples_out_h1[5]
  sig_samples_out_h1[5] = sig_samples_out_h1[4]
  sig_samples_out_h1[4] = sig_samples_out_h1[3]
  sig_samples_out_h1[3] = sig_samples_out_h1[2]
  sig_samples_out_h1[2] = sig_samples_out_h1[1]
  sig_samples_out_h1[1] = sig_samples_out_h1[0]

  sig_samples_out_h2[8] = sig_samples_out_h2[7]
  sig_samples_out_h2[7] = sig_samples_out_h2[6]
  sig_samples_out_h2[6] = sig_samples_out_h2[5]
  sig_samples_out_h2[5] = sig_samples_out_h2[4]
  sig_samples_out_h2[4] = sig_samples_out_h2[3]
  sig_samples_out_h2[3] = sig_samples_out_h2[2]
  sig_samples_out_h2[2] = sig_samples_out_h2[1]
  sig_samples_out_h2[1] = sig_samples_out_h2[0]

  sig_samples_out_h3[8] = sig_samples_out_h3[7]
  sig_samples_out_h3[7] = sig_samples_out_h3[6]
  sig_samples_out_h3[6] = sig_samples_out_h3[5]
  sig_samples_out_h3[5] = sig_samples_out_h3[4]
  sig_samples_out_h3[4] = sig_samples_out_h3[3]
  sig_samples_out_h3[3] = sig_samples_out_h3[2]
  sig_samples_out_h3[2] = sig_samples_out_h3[1]
  sig_samples_out_h3[1] = sig_samples_out_h3[0]

  sig_samples_out_h4[8] = sig_samples_out_h4[7]
  sig_samples_out_h4[7] = sig_samples_out_h4[6]
  sig_samples_out_h4[6] = sig_samples_out_h4[5]
  sig_samples_out_h4[5] = sig_samples_out_h4[4]
  sig_samples_out_h4[4] = sig_samples_out_h4[3]
  sig_samples_out_h4[3] = sig_samples_out_h4[2]
  sig_samples_out_h4[2] = sig_samples_out_h4[1]
  sig_samples_out_h4[1] = sig_samples_out_h4[0]

  sig_samples_out_h5[8] = sig_samples_out_h5[7]
  sig_samples_out_h5[7] = sig_samples_out_h5[6]
  sig_samples_out_h5[6] = sig_samples_out_h5[5]
  sig_samples_out_h5[5] = sig_samples_out_h5[4]
  sig_samples_out_h5[4] = sig_samples_out_h5[3]
  sig_samples_out_h5[3] = sig_samples_out_h5[2]
  sig_samples_out_h5[2] = sig_samples_out_h5[1]
  sig_samples_out_h5[1] = sig_samples_out_h5[0]


  # Aplicação dos filtros nas amostras
  
  sig_out_h0[sample] = \
      h0_filter_coefficients[0][0]*sig_samples_in[0] \
    + h0_filter_coefficients[0][1]*sig_samples_in[1] \
    + h0_filter_coefficients[0][2]*sig_samples_in[2] \
    + h0_filter_coefficients[0][3]*sig_samples_in[3] \
    + h0_filter_coefficients[0][4]*sig_samples_in[4] \
    - h0_filter_coefficients[1][1]*sig_samples_out_h0[1] \
    - h0_filter_coefficients[1][2]*sig_samples_out_h0[2] \
    - h0_filter_coefficients[1][3]*sig_samples_out_h0[3] \
    - h0_filter_coefficients[1][4]*sig_samples_out_h0[4]
  
  sig_out_h1[sample] = \
      h1_filter_coefficients[0][0]*sig_samples_in[0] \
    + h1_filter_coefficients[0][1]*sig_samples_in[1] \
    + h1_filter_coefficients[0][2]*sig_samples_in[2] \
    + h1_filter_coefficients[0][3]*sig_samples_in[3] \
    + h1_filter_coefficients[0][4]*sig_samples_in[4] \
    + h1_filter_coefficients[0][5]*sig_samples_in[5] \
    + h1_filter_coefficients[0][6]*sig_samples_in[6] \
    + h1_filter_coefficients[0][7]*sig_samples_in[7] \
    + h1_filter_coefficients[0][8]*sig_samples_in[8] \
    - h1_filter_coefficients[1][1]*sig_samples_out_h1[1] \
    - h1_filter_coefficients[1][2]*sig_samples_out_h1[2] \
    - h1_filter_coefficients[1][3]*sig_samples_out_h1[3] \
    - h1_filter_coefficients[1][4]*sig_samples_out_h1[4] \
    - h1_filter_coefficients[1][5]*sig_samples_out_h1[5] \
    - h1_filter_coefficients[1][6]*sig_samples_out_h1[6] \
    - h1_filter_coefficients[1][7]*sig_samples_out_h1[7] \
    - h1_filter_coefficients[1][8]*sig_samples_out_h1[8]

  sig_out_h2[sample] = \
      h2_filter_coefficients[0][0]*sig_samples_in[0] \
    + h2_filter_coefficients[0][1]*sig_samples_in[1] \
    + h2_filter_coefficients[0][2]*sig_samples_in[2] \
    + h2_filter_coefficients[0][3]*sig_samples_in[3] \
    + h2_filter_coefficients[0][4]*sig_samples_in[4] \
	  + h2_filter_coefficients[0][5]*sig_samples_in[5] \
	  + h2_filter_coefficients[0][6]*sig_samples_in[6] \
	  + h2_filter_coefficients[0][7]*sig_samples_in[7] \
	  + h2_filter_coefficients[0][8]*sig_samples_in[8] \
    - h2_filter_coefficients[1][1]*sig_samples_out_h2[1] \
    - h2_filter_coefficients[1][2]*sig_samples_out_h2[2] \
    - h2_filter_coefficients[1][3]*sig_samples_out_h2[3] \
    - h2_filter_coefficients[1][4]*sig_samples_out_h2[4] \
    - h2_filter_coefficients[1][5]*sig_samples_out_h2[5] \
    - h2_filter_coefficients[1][6]*sig_samples_out_h2[6] \
    - h2_filter_coefficients[1][7]*sig_samples_out_h2[7] \
    - h2_filter_coefficients[1][8]*sig_samples_out_h2[8]

  sig_out_h3[sample] = \
      h3_filter_coefficients[0][0]*sig_samples_in[0] \
    + h3_filter_coefficients[0][1]*sig_samples_in[1] \
    + h3_filter_coefficients[0][2]*sig_samples_in[2] \
    + h3_filter_coefficients[0][3]*sig_samples_in[3] \
    + h3_filter_coefficients[0][4]*sig_samples_in[4] \
    + h3_filter_coefficients[0][5]*sig_samples_in[5] \
    + h3_filter_coefficients[0][6]*sig_samples_in[6] \
    + h3_filter_coefficients[0][7]*sig_samples_in[7] \
    + h3_filter_coefficients[0][8]*sig_samples_in[8] \
    - h3_filter_coefficients[1][1]*sig_samples_out_h3[1] \
    - h3_filter_coefficients[1][2]*sig_samples_out_h3[2] \
    - h3_filter_coefficients[1][3]*sig_samples_out_h3[3] \
    - h3_filter_coefficients[1][4]*sig_samples_out_h3[4] \
    - h3_filter_coefficients[1][5]*sig_samples_out_h3[5] \
    - h3_filter_coefficients[1][6]*sig_samples_out_h3[6] \
    - h3_filter_coefficients[1][7]*sig_samples_out_h3[7] \
    - h3_filter_coefficients[1][8]*sig_samples_out_h3[8]

  sig_out_h4[sample] = \
      h4_filter_coefficients[0][0]*sig_samples_in[0] \
    + h4_filter_coefficients[0][1]*sig_samples_in[1] \
    + h4_filter_coefficients[0][2]*sig_samples_in[2] \
    + h4_filter_coefficients[0][3]*sig_samples_in[3] \
    + h4_filter_coefficients[0][4]*sig_samples_in[4] \
    + h4_filter_coefficients[0][5]*sig_samples_in[5] \
    + h4_filter_coefficients[0][6]*sig_samples_in[6] \
    + h4_filter_coefficients[0][7]*sig_samples_in[7] \
    + h4_filter_coefficients[0][8]*sig_samples_in[8] \
    - h4_filter_coefficients[1][1]*sig_samples_out_h4[1] \
    - h4_filter_coefficients[1][2]*sig_samples_out_h4[2] \
    - h4_filter_coefficients[1][3]*sig_samples_out_h4[3] \
    - h4_filter_coefficients[1][4]*sig_samples_out_h4[4] \
    - h4_filter_coefficients[1][5]*sig_samples_out_h4[5] \
    - h4_filter_coefficients[1][6]*sig_samples_out_h4[6] \
    - h4_filter_coefficients[1][7]*sig_samples_out_h4[7] \
    - h4_filter_coefficients[1][8]*sig_samples_out_h4[8]

  sig_out_h5[sample] = \
      h5_filter_coefficients[0][0]*sig_samples_in[0] \
    + h5_filter_coefficients[0][1]*sig_samples_in[1] \
    + h5_filter_coefficients[0][2]*sig_samples_in[2] \
    + h5_filter_coefficients[0][3]*sig_samples_in[3] \
    + h5_filter_coefficients[0][4]*sig_samples_in[4] \
    + h5_filter_coefficients[0][5]*sig_samples_in[5] \
    + h5_filter_coefficients[0][6]*sig_samples_in[6] \
    + h5_filter_coefficients[0][7]*sig_samples_in[7] \
    + h5_filter_coefficients[0][8]*sig_samples_in[8] \
    - h5_filter_coefficients[1][1]*sig_samples_out_h5[1] \
    - h5_filter_coefficients[1][2]*sig_samples_out_h5[2] \
    - h5_filter_coefficients[1][3]*sig_samples_out_h5[3] \
    - h5_filter_coefficients[1][4]*sig_samples_out_h5[4] \
    - h5_filter_coefficients[1][5]*sig_samples_out_h5[5] \
    - h5_filter_coefficients[1][6]*sig_samples_out_h5[6] \
    - h5_filter_coefficients[1][7]*sig_samples_out_h5[7] \
    - h5_filter_coefficients[1][8]*sig_samples_out_h5[8]

  # Atualização dos vetores de saída com a saída atual
  sig_samples_out_h0[0] = sig_out_h0[sample]
  sig_samples_out_h1[0] = sig_out_h1[sample]
  sig_samples_out_h2[0] = sig_out_h2[sample]
  sig_samples_out_h3[0] = sig_out_h3[sample]
  sig_samples_out_h4[0] = sig_out_h4[sample]
  sig_samples_out_h5[0] = sig_out_h5[sample]

# Cálculo dos valores RMS filtrados iterativamente
voltage_rms_h0_iterative = np.sqrt(np.mean(sig_out_h0 ** 2))
voltage_rms_h1_iterative = np.sqrt(np.mean(sig_out_h1 ** 2))
voltage_rms_h2_iterative = np.sqrt(np.mean(sig_out_h2 ** 2))
voltage_rms_h3_iterative = np.sqrt(np.mean(sig_out_h3 ** 2))
voltage_rms_h4_iterative = np.sqrt(np.mean(sig_out_h4 ** 2))
voltage_rms_h5_iterative = np.sqrt(np.mean(sig_out_h5 ** 2))

# Exibição dos valores RMS
print("Valor RMS do Nível Médio       : Manual =", "{:.3f}".format(voltage_rms_h0_iterative), " V; Função =" , "{:.3f}".format(voltage_rms_h0_measure), " V" )
print("Valor RMS da Primeira Harmônica: Manual =", "{:.3f}".format(voltage_rms_h1_iterative), "V; Função ="  , "{:.3f}".format(voltage_rms_h1_measure), "V"  )
print("Valor RMS da Segunda Harmônica : Manual =", "{:.3f}".format(voltage_rms_h2_iterative), " V; Função =" , "{:.3f}".format(voltage_rms_h2_measure), " V" )
print("Valor RMS da Terceira Harmônica: Manual =", "{:.3f}".format(voltage_rms_h3_iterative), " V; Função =" , "{:.3f}".format(voltage_rms_h3_measure), " V" )
print("Valor RMS da Quarta Harmônica  : Manual =", "{:.3f}".format(voltage_rms_h4_iterative), "  V; Função =", "{:.3f}".format(voltage_rms_h4_measure), "  V")
print("Valor RMS da Quinta Harmônica  : Manual =", "{:.3f}".format(voltage_rms_h5_iterative), "  V; Função =", "{:.3f}".format(voltage_rms_h5_measure), "  V")

Valor RMS do Nível Médio       : Manual = 11.281  V; Função = 11.281  V
Valor RMS da Primeira Harmônica: Manual = 103.974 V; Função = 103.974 V
Valor RMS da Segunda Harmônica : Manual = 21.286  V; Função = 21.286  V
Valor RMS da Terceira Harmônica: Manual = 10.505  V; Função = 10.505  V
Valor RMS da Quarta Harmônica  : Manual = 6.898   V; Função = 6.898   V
Valor RMS da Quinta Harmônica  : Manual = 5.156   V; Função = 5.156   V


## Razões para análise em frequência e processamento de sinais:

Uma questão importante que surge ao falarmos de processamento de sinais e análise em frequência é o porquê se querermos fazer essas análises e manipulações. Ou seja, por que seria interessante olhar um sinal temporal do ponto de vista de suas frequências? E por que manipular essas frequências poderia ser interessante para a ciência e para a engenharia?

A resposta se encontra na própria natureza. Mais especificamente na natureza oscilatória do mundo. Inúmeros fenômenos naturais são oscilatórios, isso é, se repetem no tempo com intervalos fixos. Esse comportamento ocorre desde fenômenos cósmicos, como o movimento do planeta terra até fenômenos microscópicos como o movimento de elétrons em um átomo. Pede-se dizer que é da natureza do universo oscilar.

O tempo necessário para que um fenômeno ocorra duas vezes é denominado período (por exemplo, a terra completa uma rotação no próprio eixo a cada dia e uma translação completa ao redor do sol a cada um ano). Chamamos de frequência a taxa em que essas oscilações ocorrem no tempo. Por exemplo: um fenômeno com um período de um segundo possui uma frequência de uma ocorrência por segundo (também chamado de Hz).

Observe que podemos definir um período em função de qualquer escala, não apenas tempo. Embora seja mais usual usar tempo como referência, um evento pode ocorrer a cada intervalo fixo de tempo, de distância, de amostras, etc.

Mas porque essas oscilações são importantes? Porque em muitos fenômenos, a frequência de sua oscilação define as características físicas do fenômeno. Por exemplo:
* A frequência da oscilação de uma massa de ar permite definir o som produzido por essa massa de ar;
* A frequência da oscilação da luz permite definir como ela será refletida e absorvida por determinados materiais e sua cor.

Além de frequência, uma oscilação pode ser definida em função de sua intensidade (magnitude) e atraso em relação a um ponto fixo (fase). A combinação de diversas oscilações, com diferentes frequências, magnitudes e fases cria infinitas possibilidades e é uma ocorrência extremamente comum.

Em outras palavras, a natureza usa oscilações como uma forma de guardar informação. Nós como seres-vivos constantemente usamos oscilações para interpretar e interagir com o mundo. Logo, nada mais natural que buscar estudar e entender a natureza dessas oscilações, e se possível, aprender a manipulá-las. No final, as ferramentas classificadas como processamento de sinais são formas de entender e manipular a natureza.

Matematicamente podemos modelar esses fenômenos oscilatórios através da função seno e da função cosseno. A função cosseno ($\cos(x)$) define um sinal que oscila do $-\infty$ até o $+\infty$ com amplitude unitária e uma frequência fixa. Já a função seno ($\sin(x)$) define praticamente o mesmo sinal, mas com uma defasagem no tempo de um quarto de oscilação (defasagem de fase de 90 graus ou $\pi$). Assim combinação das funções seno e cosseno permite a modelagem matemática de qualquer tipo de oscilação.
